<a href="https://colab.research.google.com/github/Korniev/Machine-Learning-projects/blob/main/CV/number_plate_detection_yolo5_augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PARSING DATA FROM XML AND CONVERTING IT INTO CSV

Firstly let me load all librariies wich i will us in this project at one time.As well I will using xml.etree python library to parse the data from XML and also import pandas and glob. Using glob let first get all the XML files that are produced during labeling.

In [1]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,178 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
!pip install pytesseract


In [3]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from keras.models import Model
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.applications import InceptionResNetV2
from keras.layers import Dense, Dropout, Flatten, Input
from keras.preprocessing.image import load_img, img_to_array

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q '/content/drive/My Drive/Colab Notebooks/number-plate-detection/images.zip' -d '/content/drive/My Drive/Colab Notebooks/number-plate-detection'

replace /content/drive/My Drive/Colab Notebooks/number-plate-detection/__MACOSX/._images? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [38]:
path = glob('/content/drive/My Drive/Colab Notebooks/number-plate-detection/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In the above code, we individually take each file and parse into xml.etree and find the object -> bndbox. Then we extract xmin,xmax,ymin,ymax and saved those values in the dictionary. After we convert it into a pandas data frame and save that into CSV file and save it in project folder as shown below.

In [42]:
df = pd.DataFrame(labels_dict)
df.tail()

,filepath,xmin,xmax,ymin,ymax
220,/content/drive/My Drive/Colab Notebooks/number...,217,428,147,188
221,/content/drive/My Drive/Colab Notebooks/number...,787,910,473,513
222,/content/drive/My Drive/Colab Notebooks/number...,342,494,243,288
223,/content/drive/My Drive/Colab Notebooks/number...,164,285,99,137
224,/content/drive/My Drive/Colab Notebooks/number...,1804,2493,1734,1882


We can make an augmentation creating augmented images with their xml annotations

In [41]:
import xml.etree.ElementTree as ET
from albumentations import Compose, HorizontalFlip, VerticalFlip, Rotate, Resize, BboxParams

# Функція для створення анотацій у форматі XML для аугментованих зображень
def create_augmented_annotation(original_xml, new_filepath, augmented_bboxes, image_shape):
    tree = ET.parse(original_xml)
    root = tree.getroot()

    # Оновлення інформації у XML файлі
    root.find('filename').text = os.path.basename(new_filepath)
    root.find('path').text = new_filepath
    size = root.find('size')
    size.find('width').text = str(image_shape[1])
    size.find('height').text = str(image_shape[0])
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        bbox.find('xmin').text = str(int(augmented_bboxes[0]))
        bbox.find('ymin').text = str(int(augmented_bboxes[1]))
        bbox.find('xmax').text = str(int(augmented_bboxes[2]))
        bbox.find('ymax').text = str(int(augmented_bboxes[3]))
    tree.write(new_filepath.replace('.jpeg', '.xml'))

# Конфігурація аугментацій
def get_training_augmentation():
    train_transform = [
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        Rotate(limit=45, p=0.5),
        Resize(height=512, width=512)
    ]
    return Compose(train_transform, bbox_params=BboxParams(format='pascal_voc', label_fields=['labels']))

# Функція аугментації даних
def augment_data(df, augmentor):
    augmented_data = []
    for index, row in df.iterrows():
        original_xml = row['filepath']
        image_path = original_xml.replace('.xml', '.jpeg')
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image at {image_path}")
            continue
        bboxes = [[row['xmin'], row['ymin'], row['xmax'], row['ymax'], 0]]
        augmented = augmentor(image=image, bboxes=bboxes, labels=['number_plate'])
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes'][0] if augmented['bboxes'] else [row['xmin'], row['ymin'], row['xmax'], row['ymax']]

        filename, extension = os.path.splitext(os.path.basename(image_path))
        new_filename = filename + '_aug' + extension
        new_filepath = os.path.join(os.path.dirname(image_path), new_filename)

        cv2.imwrite(new_filepath, augmented_image)
        create_augmented_annotation(original_xml, new_filepath, augmented_bboxes, augmented_image.shape)

        augmented_data.append({
            'filepath': new_filepath,
            'xmin': int(augmented_bboxes[0]),
            'ymin': int(augmented_bboxes[1]),
            'xmax': int(augmented_bboxes[2]),
            'ymax': int(augmented_bboxes[3])
        })

    return pd.DataFrame(augmented_data)

augmentor = get_training_augmentation()
augmented_df = augment_data(df, augmentor)


In [45]:
path = glob('/content/drive/My Drive/Colab Notebooks/number-plate-detection/images/*.xml')
augmented_labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    augmented_labels_dict['filepath'].append(filename)
    augmented_labels_dict['xmin'].append(xmin)
    augmented_labels_dict['xmax'].append(xmax)
    augmented_labels_dict['ymin'].append(ymin)
    augmented_labels_dict['ymax'].append(ymax)

In [52]:
augmented_df = pd.DataFrame(augmented_labels_dict)
augmented_df.to_csv('labels.csv',index=False)
augmented_df.tail()

,filepath,xmin,xmax,ymin,ymax
445,/content/drive/My Drive/Colab Notebooks/number...,171,342,201,310
446,/content/drive/My Drive/Colab Notebooks/number...,226,265,225,248
447,/content/drive/My Drive/Colab Notebooks/number...,90,220,141,199
448,/content/drive/My Drive/Colab Notebooks/number...,178,308,122,250
449,/content/drive/My Drive/Colab Notebooks/number...,101,215,379,411


As we can see, we had 224 images and their annotations.

Now thankfully to augmentation we have got 449 images with their annotations!

Well done, data is preprocessed. So we can start train our model YOLOv5

##VERIFY THE DATA

With the above code, we successfully extract the diagonal position of each image and convert the data from an unstructured to a structured format.You can have A look data above. Now also extract the respective image filename of the XML.



In [53]:
filename = augmented_df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/drive/My Drive/Colab Notebooks/number-plate-detection/images',filename_image)
    return filepath_image
getFilename(filename)

'/content/drive/My Drive/Colab Notebooks/number-plate-detection/images/N112.jpeg'

In [54]:
image_path = list(augmented_df['filepath'].apply(getFilename))
image_path[:10]#random check
len(image_path)

450

As till now we did the manual process it is important to verify the information is we got is valid or not. For that just verify the bounding box is appearing properly for a given image. Here I consider the image N2.jpeg and the corresponding diagonal position can found in df

In [55]:
file_path = image_path[449] #path of our image N2.jpeg
img = cv2.imread(file_path) #read the image
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=101, x1=215, y0=379, y1=411, xref='x', yref='y',line_color='cyan')

We can see that thankfully to augmentation we moved our image but annotation save good.

So we are ready to train our model!!

###READ DATA

This is a very important step, in this process we will take each and every image and convert it into an array using OpenCV and resize the image into 224 x 224 which is the standard compatible size of the pre-trained transfer learning model.

In [56]:
#Targeting all our values in array selecting all columns
labels = augmented_df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

After that, we will normalize the image just by dividing with maximum number as we know that the maximum number for an 8-bit image is 28 -1 = 255. That the reason we will divide our image 255.0. The way of diving an array with the maximum value is called Normalization (Min-Max Scaler). We also need to normalize our labels too. Because for the deep learning model the output range should be between 0 to 1. For normalizing labels, we need to divide the diagonal points with the width and height of the image. And finally values in a python list.

##REAL TIME NUMBER PLATE RECOGNITIONT WITH YOLOv5

Let consider the data images. Inside the data images we should how train and test. In the training test folder, we have to maintain all out images related to the training and the corresponding labels should be there in .txt format. And moreover, the label information data should consist same name as our image. Just like our training data, we will also prepare our test data.

I will write function and using that function, I can try to extract weight of the image, height of the image and also the file name from the xml files. Creating function parsing and return will be our filename, wight and height and apply all to our data frame as below.

In [58]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'/content/drive/My Drive/Colab Notebooks/number-plate-detection/images/{name}'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)

    return filename, width, height
augmented_df[['filename','width','height']] = augmented_df['filepath'].apply(parsing).apply(pd.Series)
augmented_df.tail()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
445,/content/drive/My Drive/Colab Notebooks/number...,171,342,201,310,/content/drive/My Drive/Colab Notebooks/number...,512,512
446,/content/drive/My Drive/Colab Notebooks/number...,226,265,225,248,/content/drive/My Drive/Colab Notebooks/number...,512,512
447,/content/drive/My Drive/Colab Notebooks/number...,90,220,141,199,/content/drive/My Drive/Colab Notebooks/number...,512,512
448,/content/drive/My Drive/Colab Notebooks/number...,178,308,122,250,/content/drive/My Drive/Colab Notebooks/number...,512,512
449,/content/drive/My Drive/Colab Notebooks/number...,101,215,379,411,/content/drive/My Drive/Colab Notebooks/number...,512,512


Now the next step is let me calculate the center_x, center_y, width and height, which is normalized to width and height. And as well normalize width and height of bounding box.

In [59]:
# center_x, center_y, width , height
augmented_df['center_x'] = (augmented_df['xmax'] + augmented_df['xmin'])/(2*augmented_df['width'])
augmented_df['center_y'] = (augmented_df['ymax'] + augmented_df['ymin'])/(2*augmented_df['height'])

augmented_df['bb_width'] = (augmented_df['xmax'] - augmented_df['xmin'])/augmented_df['width']
augmented_df['bb_height'] = (augmented_df['ymax'] - augmented_df['ymin'])/augmented_df['height']
augmented_df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,/content/drive/My Drive/Colab Notebooks/number...,179,347,256,290,/content/drive/My Drive/Colab Notebooks/number...,500,344,0.526000,0.793605,0.336000,0.098837
1,/content/drive/My Drive/Colab Notebooks/number...,137,262,249,290,/content/drive/My Drive/Colab Notebooks/number...,610,485,0.327049,0.555670,0.204918,0.084536
2,/content/drive/My Drive/Colab Notebooks/number...,150,378,217,269,/content/drive/My Drive/Colab Notebooks/number...,500,393,0.528000,0.618321,0.456000,0.132316
3,/content/drive/My Drive/Colab Notebooks/number...,207,356,174,287,/content/drive/My Drive/Colab Notebooks/number...,441,363,0.638322,0.634986,0.337868,0.311295
4,/content/drive/My Drive/Colab Notebooks/number...,39,108,129,157,/content/drive/My Drive/Colab Notebooks/number...,400,300,0.183750,0.476667,0.172500,0.093333


##Load model and Split data

One of the major challenges in the training of YOLO is hardware itself. YOLO requires a fast-processing chip used to get trained. It is extremely difficult to train the YOLO in normal CPUs. Firstly, we have to clone YOLOV5 to our work space. You can find it here and install all requirements. Make sure you in right folder type as below, let's do it.

In [60]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 16582 (delta 30), reused 39 (delta 18), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.13 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (11387/11387), done.


In [ ]:
!pip install -r ./yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

We need to make a one more folder and make into it two more - name it as data_images and train and test. We will split data to train and split

In [ ]:
!pip install pillow==10.0.1


In [61]:
!mkdir "/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/"

In [62]:
!mkdir "/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/test/"

In [63]:
!mkdir "/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/train/"

In [64]:
### split the data into train and test
df_train = augmented_df.iloc[:360]
df_test = augmented_df.iloc[360:]

We will copy each image into the folder train and test and generate .txt which has label info.

In [65]:
train_folder = '/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/train/'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')

    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()

test_folder = '/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/test/'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')

    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()

As well I will create data.yaml file with path to train and validation, as well number of classes as 1, name it in list as license plate. You will need to save this in your dataset because we will need to provide the path on it later.

##TRAINING YOLO

Next step is basically train model. It could take time be ready for it. You can use Kaggle or GoogleColab for it. We already set out data.yaml file, lets us give path to it and train model. Below you will find code to cleare you GPU(It helped me a lot and will save you time.)

In [66]:
!pip install GPUtil


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=d84c85c0becfbcf61bea07b16fe9de13783825f409fe85e890e4aaa9dda9049d
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [67]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  3% |  1% |


In [70]:
!python ./yolov5/train.py --data "/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/data.yaml" --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model_plate_recognition --epochs 100

2024-05-07 13:17:49.902512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 13:17:49.902577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 13:17:49.903935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=/content/drive/My Drive/Colab Notebooks/number-plate-detection/data_images/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/

When model trained, we need to save our it in order to use it at OCR in onnx format as below:

In [72]:
!python ./yolov5/export.py --weight ./yolov5/runs/train/Model_plate_recognition2/weights/best.pt --include torchscript onnx

export: data=yolov5/data/coco128.yaml, weights=['./yolov5/runs/train/Model_plate_recognition2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v7.0-307-g920c721e Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/Model_plate_recognition2/weights/best.pt with output shape (1, 25200, 6) (13.8 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 2.9s, saved as yolov5/runs/train/Model_plate_recognition2/weights/best.torchscript (27.2 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     

##PREDICTING RESULTS FROM YOLO

Let’s do some images settings which is omportant to provide in model construction. Input settings we need to pass as 640 by 640

In [73]:
# settings
INPUT_WIDTH =  640
INPUT_HEIGHT = 640

In [81]:
# LOAD THE IMAGE
img = io.imread('/content/drive/MyDrive/Colab Notebooks/number-plate-detection/A.png')

fig = px.imshow(img)
fig.update_layout(width=700, height=400, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

Now, we will load our YOLO model.

In [82]:
# LOAD YOLO MODEL
net = cv2.dnn.readNetFromONNX('./yolov5/runs/train/Model_plate_recognition2/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

The next step is I need to parse this image to my YOLO model and get the prediction. Important - size we need to set as input width a 640 and input height is 640.

We have the 25200 rows and 6 columns. The first four columns will give you the information about the bounding box, which is center X and, Center Y, W and H. And these values are normalized to 640 by 640. And the next two values are the confidence and the probabilities. Confidence tells us what is the confidence for the detecting the bounding box and the next one is the probability score of the class, since we have only one class that is number plate.

Let’s make a list of boxes and confidence and place it order. As well we need to identify our X_factor and Y_factor of image.

We have to filter detections based on the confidence score and the probability score. Let me take the confidence >0.4 and filter from class >0.25.

There is will be one of major issue with YOLO modeling it will give you the repeated boxes, but the same object it might give you that multiple boxes. For that we need to perform the non-maximum suppression. To do it we need to provide all our clean boxes and confidences.

In the end of that part of code I will take bounding boxes and draw the rectangle box to our image. And will write it all in functions.

In [83]:
def get_detections(img,net):
    # 1.CONVERT IMAGE TO YOLO FORMAT
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    # 2. GET PREDICTION FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]

    return input_image, detections

def non_maximum_supression(input_image,detections):

    # 3. FILTER DETECTIONS BASED ON CONFIDENCE AND PROBABILIY SCORE

    # center x, center y, w , h, conf, proba
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detecting license plate
        if confidence > 0.4:
            class_score = row[5] # probability score of license plate
            if class_score > 0.25:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])


                confidences.append(confidence)
                boxes.append(box)

    # 4.1 CLEAN
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    # 4.2 NMS
    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)

    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    # 5. Drawings
    for ind in index:
        x,y,w,h =  boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        license_text = extract_text(image,boxes_np[ind])


        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+25),(0,0,0),-1)


        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,license_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)

    return image

In [84]:
# predictions flow with return result
def yolo_predictions(img,net):
    # step-1: detections
    input_image, detections = get_detections(img,net)
    # step-2: NMS
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)
    # step-3: Drawings
    result_img = drawings(img,boxes_np,confidences_np,index)
    return result_img

Here we will not only detect license plate but else extract test from it. We already done it before let’s repeat it again. We will write function. First from bounding box we need to take our x,y,w,h and extract ROI. I place it at top of our functions in drawing part.

In [85]:
# extrating text
def extract_text(image,bbox):
    x,y,w,h = bbox
    roi = image[y:y+h, x:x+w]

    if 0 in roi.shape:
        return 'no number'

    else:
        text = pt.image_to_string(roi)
        text = text.strip()

        return text

In [87]:
# test
img = io.imread('/content/drive/MyDrive/Colab Notebooks/number-plate-detection/A.png')
results = yolo_predictions(img,net)

In [88]:
fig = px.imshow(img)
fig.update_layout(width=800, height=700, margin=dict(l=10, r=10, b=10, t=10))
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()